In [1]:
from src.utils.database_utils import create_client, search_documents
import torch
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Yangliang
[nltk_data]     Lu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Yangliang
[nltk_data]     Lu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(device)

cuda


In [3]:
client = create_client()

In [4]:
query = {
        "match": {
            "keywords": "Bloomberg"
        }
    }

In [5]:
res = search_documents(client, "documents", query)

In [6]:
def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text)

    # Initialize the stemmer
    stemmer = PorterStemmer()

    # Remove stopwords and apply stemming
    filtered_words = [stemmer.stem(word) for word in words if word.lower() not in set(stopwords.words('english'))]

    # Join the filtered words back into a string
    preprocessed_text = ' '.join(filtered_words)

    return preprocessed_text

In [7]:
# concatenate all the text to one string
text = ""
for doc in res["hits"]["hits"]:
    text += doc["_source"]["body"] + '\n'

In [19]:
text_list = [doc["_source"]["body"] for doc in res["hits"]["hits"]]

In [8]:
from transformers import pipeline
hf_name = 'pszemraj/led-large-book-summary'

summarizer = pipeline(
    "summarization",
    hf_name,
    device=0 if torch.cuda.is_available() else -1,
)

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [20]:
result = summarizer(
    text_list,
    min_length=16,
    max_length=256,
    no_repeat_ngram_size=3,
    encoder_no_repeat_ngram_size=3,
    repetition_penalty=3.5,
    num_beams=4,
    early_stopping=True,
    clean_up_tokenization_spaces = True,
)

In [21]:
result

[{'summary_text': "In a test of investor interest in bitcoin, the cryptocurrency briefly surges 10% on Monday before paring its gains Tuesday. That day, some investors liquidate positions that were built up anticipating lower prices. There is little buyer interest in the cryptocoins after last year's rout and other recent market crashes. This week, however, there is some speculation about whether the SEC will eventually approve the launch of exchange traded funds that offer to buy and sell Bitcoins on the open market. On Tuesday, the CBOE Volatility index hits an all-time low. In the aftermath of the crash, some speculators began adding to their bearish bets on the currency. On Wednesday, the VIX trades as high as 2023, almost double the level at which it closed the day before. Traders speculate on whether this week's strong economic data will lead to greater adoption of cryptos."},
 {'summary_text': 'Bond traders get a glimpse "of what the future end might look like." The Federal Rese

In [16]:
result = summarizer(
    text,
    min_length=16,
    max_length=64,
    no_repeat_ngram_size=3,
    encoder_no_repeat_ngram_size=3,
    repetition_penalty=3.5,
    num_beams=4,
    early_stopping=True,
    clean_up_tokenization_spaces = True,
)

In [17]:
result

[{'summary_text': "In this week's edition of the Market Wrap, we get a brief glimpse of what could happen to the cryptocurrency market as the Securities and Exchange Commission prepares to decide on whether or not to approve exchange traded funds that would allow investors to buy and sell directly into the currency. A rumor that Blackrock has"}]